To cluster the documents and annotate the clusters according to the main topic, we can use various natural language 
processing and machine learning techniques. Here is a general framework that can be used to accomplish this task:

1.Load the dataset into a pandas dataframe.
2.Clean the data by removing any null values or duplicates.
3.Preprocess the textual data (i.e., the abstracts) by removing stopwords, punctuation, and other noise. This can be done using various libraries like NLTK or spaCy.
4.Convert the preprocessed abstracts into a numerical representation using a technique like TF-IDF or Doc2Vec.
5. Find the relevant document 
6.To find the main topic of each document, we will use a topic modeling algorithm such as Latent Dirichlet Allocation (LDA) or Non-negative Matrix Factorization (NMF)
6.For clustering and visualization, we will use a dimensionality reduction technique such as Principal Component Analysis (PCA) or t-SNE, and a visualization library such as bokeh.''' 

In [19]:
'''Packages for preprocessing'''
import nltk 
from nltk.tokenize import word_tokenize,sent_tokenize

'''Pakages to load dataset'''
import pandas as pd

import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.decomposition import NMF
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

from bokeh.plotting import figure, show, output_file, ColumnDataSource
from bokeh.models import HoverTool


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 


nltk.download('punkt')
nltk.download('stopwords')

from sklearn.metrics.pairwise import cosine_similarity



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sitas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sitas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
data = pd.read_csv("Data_mesh_cleaned.csv")
pdf_data = data.copy()


In [25]:
def NMF_modeling(df_data,components):
    
    df = df_data.copy()
    
 
    model = TfidfVectorizer(stop_words='english', max_df = 0.5, ngram_range=(2,3))
    word_vector = normalize(model.fit_transform(df['pdf_content']))
    nmf_model = NMF(n_components= components, init='nndsvd')
    nmf_model.fit(word_vector)
    df['topic'] = nmf_model.transform(word_vector).argmax(axis=1)
    
    topic_words = []
    n_words = 10
    
    feature_names = model.get_feature_names_out()
    for topic_idx, topic in enumerate(nmf_model.components_):
        top_words_idx = topic.argsort()[:-n_words - 1:-1]
        top_words = [feature_names[i] for i in top_words_idx]
        topic_words.append(', '.join(top_words))

    df['topic_words'] = [topic_words[i] for i in df['topic']]
    
    # Transform the topic_words column into a TF-IDF matrix
    tfidf_matrix = model.transform(df['topic_words'])
    dominant_words = []
    for i in range(len(df)):
        row = tfidf_matrix[i]
        dominant_word_idx = row.argmax()
        dominant_word = model.get_feature_names_out()[dominant_word_idx]
        dominant_words.append(dominant_word)

# Add the dominant_word column to the DataFrame
    df['dominant_word'] = dominant_words
    
    return df

In [16]:
from sklearn.decomposition import LatentDirichletAllocation

def LDA_modeling(df_data, components):
    df = df_data.copy()

    # Create a TF-IDF vectorizer
    vectorizer = TfidfVectorizer(stop_words='english')
    word_vector = vectorizer.fit_transform(df['pdf_content'])

    # Fit an LDA model
    lda_model = LatentDirichletAllocation(n_components=components, random_state=42)
    lda_model.fit(word_vector)

    # Assign topics to documents
    df['topic'] = lda_model.transform(word_vector).argmax(axis=1)

    # Get top words for each topic
    topic_words = []
    n_words = 10
    feature_names = vectorizer.get_feature_names_out()
    
    for topic_idx, topic in enumerate(lda_model.components_):
        top_words_idx = topic.argsort()[:-n_words - 1:-1]
        top_words = [feature_names[i] for i in top_words_idx]
        topic_words.append(', '.join(top_words))

    df['topic_words'] = [topic_words[i] for i in df['topic']]

 
    tfidf_matrix = vectorizer.transform(df['topic_words'])
    
    
    dominant_words = []
    for i in range(len(df)):
        row = tfidf_matrix[i]
        dominant_word_idx = row.argmax()
        dominant_word = feature_names[dominant_word_idx]
        dominant_words.append(dominant_word)
    df['dominant_word'] = dominant_words

    return df



In [24]:
result = NMF_modeling(pdf_data, components = 20)

In [26]:
result

,pdf_name,pdf_content,topic,topic_words,dominant_word
0,0670.pdf,introducing paradigm smart city platform desig...,13,"smart city, digital twin, user scenario, numbe...",smart city
1,1-s2.0-S1877050921022365-main.pdf,available procedia computer science author pub...,5,"smart monitoring, event based, event based sma...",smart monitoring
2,1-s2.0-S1877050923006099-main.pdf,available procedia computer science author pub...,5,"smart monitoring, event based, event based sma...",smart monitoring
3,2302.01713.pdf,best avoid mess jan bode ibm germany niklas kü...,2,"http doi, http doi org, doi org, challenge bes...",http doi
4,2304.01062.pdf,systematic gray literature review abel goedege...,0,"gray literature, platform service, manuscript ...",composite product
5,978-1-4842-9253-2.pdf,fabric approach ai guide ai based cataloging g...,15,"knowledge catalog, ai ml, chapter fabri, hybri...",ai governance
6,978-3-031-12423-5_7.pdf,cok survey privacy challenge relation mesh nik...,18,"quasi identi, quasi identi er, identi er, eren...",erential privacy
7,978-3-031-36118-0.pdf,lecture note engineering communication technol...,6,"di process, http doi org, http doi, doi org, i...",http doi
8,978-3-031-39847-6_1.pdf,integration revitalized warehouse lake robert ...,6,"di process, http doi org, http doi, doi org, i...",http doi
9,978-3-031-45021-1_23.pdf,converging microservice principle uni ed logic...,10,"microservice principle, bounded context, morai...",morais et


,Document Title,Year,Abstract,PDF Link,label,topic,topic_words,dominant_word
0,diagnostic maintenance technique using computer,1963,possible technique attending software need adv...,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,0,3,"test, testing, case, coverage, suite",case
1,high speed electro optical mechanical phototyp...,1968,adanced performance automated phototypesetting...,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,0,1,"network, algorithm, performance, power, data",algorithm
2,recognition handprinted numeral two stage feat...,1970,optical character recognition system handprint...,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,0,1,"network, algorithm, performance, power, data",algorithm
3,computer diagnosis using blocking gate approach,1971,previous paper author considered application g...,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,0,2,"fault, reliability, failure, tolerant, detection",detection
4,simulation modeling air quality control,1971,simulation modeling major role air quality pro...,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,0,0,"software, metric, model, quality, defect",defect
...,...,...,...,...,...,...,...,...
5995,research relationship curvature radius deflect...,2011,china current specification design asphalt pav...,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,0,1,"network, algorithm, performance, power, data",algorithm
5996,sampling dmr practical low overhead permanent ...,2011,technology scaling manufacture time field perm...,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,0,2,"fault, reliability, failure, tolerant, detection",detection
5997,qos aware multipath routing protocol delay sen...,2011,paper proposes qos multipath routing protocol ...,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,0,1,"network, algorithm, performance, power, data",algorithm
5998,monitoring high performance data stream vertic...,2011,last several year monitoring high performance ...,http://ieeexplore.ieee.org/stamp/stamp.jsp?arn...,0,1,"network, algorithm, performance, power, data",algorithm


# 